In [269]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
#getting a sense of the data
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary

In [270]:
# Calculate the total number of schools
total_school = school_data_complete["School ID"].nunique()

# Calculate the total number of students total_students = school_data_complete['Student ID'].sum()
total_students = school_data_complete["Student ID"].count()
total_students_format = "{:,}".format(total_students)

# Calculate the total budget 
school_group = school_data_complete.groupby("School ID")
budget_by_school = school_group.mean()
budget_sum = budget_by_school["budget"].sum()
budget_sum_format = "${:,}".format(budget_sum)

# Calculate the average math score
mean_math = school_data_complete["math_score"].mean()
mean_math_format = "{0:.2f}".format(mean_math)

# Calculate the average reading score
mean_read = school_data_complete["reading_score"].mean()
mean_read_format = "{0:.2f}".format(mean_read)

# Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2
avg_passing_rate = (mean_math+mean_read)/2
avg_passing_rate_format = "{0:.2f}%".format(avg_passing_rate)

# Calculate the percentage of students with a passing math score (70 or greater)

perc_pass_math = ((school_data_complete[school_data_complete["math_score"]>=70].count())/total_students)*100
perc_pass_math = perc_pass_math["math_score"]
perc_pass_math_format = "{0:.2f}%".format(perc_pass_math)

# Calculate the percentage of students with a passing reading score (70 or greater)
perc_pass_read = ((school_data_complete[school_data_complete["reading_score"]>=70].count())/total_students)*100
perc_pass_read = perc_pass_read["reading_score"]
perc_pass_read_format = "{0:.2f}%".format(perc_pass_read)

#---PREPARING PASS/FAIL MATH AND READ FOR NEXT EXERCISE ----
school_data_complete["passed math count"] = np.where(school_data_complete["math_score"]>=70, 1,0)
school_data_complete["passed read count"] = np.where(school_data_complete["reading_score"]>=70, 1,0)


# Create a dataframe to hold the above results
district_df = { "Total Schools" : [total_school], "Total Students" : [total_students_format], 
               "Total Budget" :[budget_sum_format], "Average Math Score": [mean_math_format],
               "Average Read Score":[mean_read_format],  '% Passing Math' : [perc_pass_math_format],
               '% Passing Read' : [perc_pass_read_format], "% Overall Passing": [avg_passing_rate_format]}
district_df = pd.DataFrame(district_df)
district_df
# Optional: give the displayed data cleaner formatting


,Total Schools,Total Students,Total Budget,Average Math Score,Average Read Score,% Passing Math,% Passing Read,% Overall Passing
0,15,"39,170","$24,649,428.0",78.99,81.88,74.98%,85.81%,80.43%


# School Summary

In [283]:
# Create an overview table that summarizes key metrics about each school, including:

school_name_group_df = school_data_complete.groupby(["school_name"]).agg(
    {
        'type': "max",
        'Student ID': "count",
        'budget': "mean",
        'math_score' : "mean",
        'reading_score' : "mean",
        'passed math count' : "sum",
        'passed read count' : "sum",
    }
)

#adding new columns with calculations based on the aggregations above
school_name_group_df["student_per_budget"] = school_name_group_df['budget']/school_name_group_df['Student ID']
school_name_group_df["% Passing Math"] = (school_name_group_df['passed math count']/school_name_group_df['Student ID'])*100
school_name_group_df["% Passing Read"] = (school_name_group_df['passed read count']/school_name_group_df['Student ID'])*100
school_name_group_df["% Overall Passing Rate"] =(school_name_group_df["% Passing Math"]+school_name_group_df["% Passing Read"])/2

# just renaming column
school_name_group_df["Student Count"] = school_name_group_df["Student ID"]

# deleting columns I don't need to show
del school_name_group_df["passed math count"]
del school_name_group_df["passed read count"]
del school_name_group_df["Student Count"]

# sort for top 5 performers (overall passing rate)
school_name_group_df = school_name_group_df.sort_values(by="% Overall Passing Rate", ascending=False)

school_name_group_df.head(5)

,type,Student ID,budget,math_score,reading_score,student_per_budget,% Passing Math,% Passing Read,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,83.061895,83.975780,582.0,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,83.418349,83.848930,638.0,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,83.839917,84.044699,609.0,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,83.351499,83.816757,625.0,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,83.274201,83.989488,578.0,93.867718,96.539641,95.203679


Top 5 performing schools (based on Overall Passing Rate)


## Bottom 5 Performing Schools (By Passing Rate)

In [272]:
# Sort and display the bottom five schools in overall passing rate
school_name_group_df_worse = school_name_group_df.sort_values(by="% Overall Passing Rate", ascending=True)
school_name_group_df_worse.head(5)

,type,Student ID,budget,math_score,reading_score,student_per_budget,% Passing Math,% Passing Read,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,76.842711,80.744686,637.0,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411,76.711767,81.158020,639.0,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635,76.629414,81.182722,655.0,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,77.072464,80.966394,650.0,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916,77.102592,80.746258,644.0,68.309602,79.299014,73.804308


## Reading Scores by Grade

In [281]:
# Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th)
#at each school.

school_grade_group_df = school_data_complete.groupby(["school_name", "grade"])
school_grade_group_df = school_grade_group_df['reading_score'].mean()
school_grade_group_df =  pd.DataFrame(school_grade_group_df)
school_grade_group_df = school_grade_group_df.reset_index()

school_grade_pivot = school_grade_group_df.pivot(index='school_name', columns='grade', values='reading_score')
school_grade_pivot = school_grade_pivot.style.format("{:,.2f}")
school_grade_pivot



grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,80.91,80.95,80.91,81.30
Cabrera High School,84.25,83.79,84.29,83.68
Figueroa High School,81.41,80.64,81.38,81.20
Ford High School,81.26,80.40,80.66,80.63
Griffin High School,83.71,84.29,84.01,83.37
Hernandez High School,80.66,81.40,80.86,80.87
Holden High School,83.32,83.82,84.70,83.68
Huang High School,81.51,81.42,80.31,81.29
Johnson High School,80.77,80.62,81.23,81.26


## Math Scores by Grade

In [280]:
# Perform the same operations as above for math scores
school_grade_group_df2 = school_data_complete.groupby(["school_name", "grade"])
school_grade_group_df2 = school_grade_group_df2['math_score'].mean()
school_grade_group_df2 =  pd.DataFrame(school_grade_group_df2)
school_grade_group_df2 = school_grade_group_df2.reset_index()

school_grade_pivot2 = school_grade_group_df2.pivot(index='school_name', columns='grade', values='math_score')

school_grade_pivot2 = school_grade_pivot2.style.format("{:,.2f}")
school_grade_pivot2


grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,77.00,77.52,76.49,77.08
Cabrera High School,83.15,82.77,83.28,83.09
Figueroa High School,76.54,76.88,77.15,76.40
Ford High School,77.67,76.92,76.18,77.36
Griffin High School,84.23,83.84,83.36,82.04
Hernandez High School,77.34,77.14,77.19,77.44
Holden High School,83.43,85.00,82.86,83.79
Huang High School,75.91,76.45,77.23,77.03
Johnson High School,76.69,77.49,76.86,77.19


## Scores by School Spending

In [296]:
Size_bins# Create a table that breaks down school performances based on average Spending Ranges (Per Student).
# Use 4 reasonable bins to group school spending. Include in the table each of the following:
budget_bins = [0, 585, 615, 645, 675]
budget_labels = ["Small <585", "Medium <615", "High <645", "Top - up tp 675"]

school_name_group_df["Budget Level per Student"] = pd.cut(school_name_group_df["student_per_budget"], budget_bins, labels=budget_labels)
school_name_group_df_bugdet_bins = school_name_group_df.groupby("Budget Level per Student")

Budget_bins = school_name_group_df_bugdet_bins.mean()
Budget_bins = pd.DataFrame(Budget_bins)

Budget_bins = Budget_bins.rename(index=str, columns={"math_score": "Avg. Math Score", "reading_score": "Avg. Reading Score"})

del Budget_bins['Student ID']
del Budget_bins['budget']
del Budget_bins['student_per_budget']

Budget_bins = Budget_bins.style.format("{:,.2f}")

Budget_bins


,Avg. Math Score,Avg. Reading Score,% Passing Math,% Passing Read,% Overall Passing Rate
Budget Level per Student,,,,,
Small <585,83.46,83.93,93.46,96.61,95.04
Medium <615,83.60,83.89,94.23,95.90,95.07
High <645,79.08,81.89,75.67,86.11,80.89
Top - up tp 675,77.00,81.03,66.16,81.13,73.65



# Scores by School Size

In [278]:
# Scores by School Size
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_name_group_df["School Size"] = pd.cut(school_name_group_df["Student ID"], size_bins, labels=group_names)
school_name_group_df_size_bins = school_name_group_df.groupby("School Size")

Size_bins = school_name_group_df_size_bins.mean()
Size_bins = pd.DataFrame(Size_bins)
Size_bins = Size_bins.rename(index=str, columns={"math_score": "Avg. Math Score", "reading_score": "Avg. Reading Score"})

del Size_bins['Student ID']
del Size_bins['budget']
del Size_bins['student_per_budget']

Size_bins = Size_bins.style.format("{:,.2f}")

Size_bins


# Perform the same operations as above, based on school size.

,Avg. Math Score,Avg. Reading Score,% Passing Math,% Passing Read,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.82,83.93,93.55,96.10,94.82
Medium (1000-2000),83.37,83.86,93.60,96.79,95.20
Large (2000-5000),77.75,81.34,69.96,82.77,76.36


# Scores by School Type

In [277]:
# Scores by School Type
# Perform the same operations as above, based on school type.
school_type_group_df = school_name_group_df.groupby('type')
school_type_group_df = school_type_group_df.mean()
school_type_group_df =  pd.DataFrame(school_type_group_df)

del school_type_group_df['Student ID']
del school_type_group_df['budget']
del school_type_group_df['student_per_budget']
school_type_group_df = school_type_group_df.rename(index=str, columns={"math_score": "Avg. Math Score", "reading_score": "Avg. Reading Score"})
school_type_group_df = school_type_group_df.style.format("{:,.2f}")
school_type_group_df

,Avg. Math Score,Avg. Reading Score,% Passing Math,% Passing Read,% Overall Passing Rate
type,,,,,
Charter,83.47,83.90,93.62,96.59,95.10
District,76.96,80.97,66.55,80.80,73.67


## Analysis:

    
    When looking at the top 5 and bottom 5 performing schools, we already see a pattern appearing regarding the school type: All top 5 schools where charter and all bottom 5 were District schools. Interestingly, the top performing school (Cabrera) falls into the small budget/student category. It is also observed that schools that fall under the highets budget level/student have the lowest overall passing rate (73%)
    Although the difference of passing scores between Small and Medium size schools is not significant (94% versus 95% respectively), Large schools much lower overall passing score (76%) with math subject bringing the average down. 
    This analysis indicates that, for this specific district, the best schools tend to be Charter schools, with fewer than 2,000 students with budget/student around $600. 